---
title: "Medical Assitant"
date: 2025-08-05
---

## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [311]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install --no-cache-dir llama-cpp-python==0.2.77 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu124


In [312]:
!pip install huggingface_hub pandas tiktoken pymupdf langchain langchain-community chromadb sentence-transformers "numpy<3" -q

In [313]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## Question Answering using LLM

#### Downloading and Loading the model

In [314]:
## Model configuration
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf"
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
    )

In [315]:
#uncomment the below snippet of code if the runtime is connected to GPU.
llm = Llama(
    model_path=model_path,
    n_ctx=4096,
    n_gpu_layers=38,
    n_batch=512
)


llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_

#### Response

In [316]:
def response(query, max_tokens=512, temperature=0, top_p=1.0, top_k=0):

    model_output = llm(
        prompt=query,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k
    )

    return model_output['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [317]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
response(user_prompt)


llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     329.67 ms /   512 runs   (    0.64 ms per token,  1553.08 tokens per second)
llama_print_timings: prompt eval time =    1761.76 ms /    17 tokens (  103.63 ms per token,     9.65 tokens per second)
llama_print_timings:        eval time =   17446.56 ms /   511 runs   (   34.14 ms per token,    29.29 tokens per second)
llama_print_timings:       total time =   20087.38 ms /   528 tokens


"\nSepsis is a life-threatening condition that can arise from an infection, and it is a leading cause of death in hospitals. The management of sepsis in a critical care unit (CCU) requires a systematic approach that includes early recognition, prompt treatment, and ongoing monitoring. Here is a general protocol for managing sepsis in a CCU:\n\n1. Early recognition:\n\t* Use a standardized screening tool, such as the Sepsis-3 definition, to identify patients who are at risk of developing sepsis.\n\t* Monitor patients for signs and symptoms of sepsis, such as fever, tachycardia, tachypnea, and altered mental status.\n2. Initial assessment:\n\t* Perform a thorough physical examination to identify any signs of infection or organ dysfunction.\n\t* Order blood cultures and other diagnostic tests (such as complete blood count, electrolytes, and renal function tests) to confirm the diagnosis of sepsis.\n3. Resuscitation:\n\t* Administer broad-spectrum antibiotics within the first hour of recog

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [318]:
user_prompt = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response(user_prompt)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     314.81 ms /   490 runs   (    0.64 ms per token,  1556.51 tokens per second)
llama_print_timings: prompt eval time =     238.00 ms /    33 tokens (    7.21 ms per token,   138.66 tokens per second)
llama_print_timings:        eval time =   16052.19 ms /   489 runs   (   32.83 ms per token,    30.46 tokens per second)
llama_print_timings:       total time =   17097.09 ms /   522 tokens


'\n\nAnswer:\n\nAppendicitis is a medical emergency that requires prompt treatment. The most common symptoms of appendicitis include:\n\n1. Severe pain in the abdomen, usually starting near the belly button and then moving to the lower right side of the abdomen.\n2. Nausea and vomiting.\n3. Loss of appetite.\n4. Fever.\n5. Abdominal tenderness and guarding (muscle tension).\n6. Abdominal swelling.\n7. Diarrhea or constipation.\n\nIf you suspect that you or someone else may have appendicitis, it is essential to seek medical attention immediately. Appendicitis can be cured via surgery, but not with medicine. In fact, delaying treatment can lead to serious complications, such as the appendix rupturing and spreading infection throughout the abdominal cavity.\n\nThe surgical procedure used to treat appendicitis is called an appendectomy. There are two types of appendectomies: open and laparoscopic.\n\n1. Open Appendectomy: This is the traditional method, where a single incision is made in t

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [319]:
user_prompt = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response(user_prompt)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     322.16 ms /   512 runs   (    0.63 ms per token,  1589.28 tokens per second)
llama_print_timings: prompt eval time =     247.18 ms /    37 tokens (    6.68 ms per token,   149.69 tokens per second)
llama_print_timings:        eval time =   16779.47 ms /   511 runs   (   32.84 ms per token,    30.45 tokens per second)
llama_print_timings:       total time =   17860.28 ms /   548 tokens


"\nSudden patchy hair loss, also known as alopecia areata, can be caused by a variety of factors. Here are some effective treatments and solutions for addressing this condition:\n\n1. Corticosteroid injections: These injections can help suppress the immune system and promote hair growth. They are usually given every 4-6 weeks and can be effective in promoting hair regrowth.\n2. Topical corticosteroids: Over-the-counter or prescription topical corticosteroids can be applied directly to the affected area to reduce inflammation and promote hair growth.\n3. Minoxidil (Rogaine): This is a solution that is applied to the scalp to stimulate hair growth. It is available over-the-counter and can be effective in promoting hair regrowth.\n4. Anthralin: This is a medication that is applied to the skin to reduce inflammation and promote hair growth. It is often used in combination with corticosteroids.\n5. Phototherapy: Exposure to specific wavelengths of light, such as ultraviolet B (UVB) or narro

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [320]:
user_prompt = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response(user_prompt)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     315.46 ms /   512 runs   (    0.62 ms per token,  1623.05 tokens per second)
llama_print_timings: prompt eval time =     370.89 ms /    31 tokens (   11.96 ms per token,    83.58 tokens per second)
llama_print_timings:        eval time =   16520.01 ms /   511 runs   (   32.33 ms per token,    30.93 tokens per second)
llama_print_timings:       total time =   17724.66 ms /   542 tokens


"\nThere are several treatments that may be recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function. The specific treatment plan will depend on the location and severity of the injury, as well as the individual's overall health and medical history. Some common treatments for brain injuries include:\n\n1. Medications: To manage symptoms such as pain, inflammation, and anxiety.\n2. Rehabilitation therapy: To help regain lost function and improve cognitive, physical, and emotional abilities. This may include physical therapy, occupational therapy, speech therapy, and cognitive therapy.\n3. Surgery: To repair damaged brain tissue or relieve pressure on the brain.\n4. Neurostimulation therapies: Such as transcranial magnetic stimulation (TMS) or transcranial direct current stimulation (tDCS), which can help improve cognitive function and reduce symptoms of depression.\n5. Cognitive rehabilitation: To help

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [321]:
user_prompt = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response(user_prompt)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     317.25 ms /   512 runs   (    0.62 ms per token,  1613.87 tokens per second)
llama_print_timings: prompt eval time =     224.85 ms /    37 tokens (    6.08 ms per token,   164.55 tokens per second)
llama_print_timings:        eval time =   16845.06 ms /   511 runs   (   32.96 ms per token,    30.34 tokens per second)
llama_print_timings:       total time =   17914.58 ms /   548 tokens


'\n\nA person who has fractured their leg during a hiking trip requires prompt medical attention to ensure proper healing and prevent complications. Here are some necessary precautions and treatment steps:\n\n1. Stop all activity: The first step is to stop all activity and rest the affected leg immediately. This will help prevent further damage and reduce the risk of complications.\n2. Assess the severity of the injury: Evaluate the extent of the fracture and determine if there are any other injuries that require attention.\n3. Immobilize the leg: Use a splint or brace to immobilize the affected leg and prevent further movement. This will help reduce pain and promote healing.\n4. Apply ice: Apply ice packs to the affected area to reduce swelling and pain. Wrap the ice in a towel or cloth to avoid direct contact with the skin.\n5. Elevate the leg: Elevate the affected leg above the level of the heart to reduce swelling and promote blood flow.\n6. Monitor for signs of complications: Moni

## Question Answering using LLM with Prompt Engineering

In [322]:
from IPython.display import Markdown, display

system_prompt = """
You are a highly knowledgeable clinical assistant trained in evidence-based medicine. Your job is to answer clinical questions clearly, accurately, and in a structured format suitable for healthcare professionals or informed patients.

Follow these rules:
- Use **bold section titles**
- Break answers into logical sections: causes, symptoms, diagnosis, treatment, recovery, etc.
- Use numbered or bulleted lists where appropriate
- Use concise medical language—short, clear sentences
- Do not speculate. If something is unknown or controversial, say so.

If the question involves treatment, list both **medical** and **surgical** options if relevant.
If the condition has subtypes or multiple causes, include them.
If the treatment requires further testing or depends on context, mention that.

Answer the following question:
"""


def response_prompt(query, max_tokens=1024, temperature=0, top_p=1.0, top_k=0):
    prompt = system_prompt + f"### Question:\n{query}\n\n### Answer:\n"

    model_output = llm(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k
    )

    return model_output['choices'][0]['text']

def display_markdown_output(raw_text):
    cleaned_text = raw_text.encode().decode('unicode_escape').strip()
    display(Markdown(cleaned_text))

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [323]:
user_prompt = "What is the protocol for managing sepsis in a critical care unit?"
raw_response = response_prompt(user_prompt)
display_markdown_output(raw_response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     555.17 ms /   875 runs   (    0.63 ms per token,  1576.10 tokens per second)
llama_print_timings: prompt eval time =     373.61 ms /   214 tokens (    1.75 ms per token,   572.79 tokens per second)
llama_print_timings:        eval time =   28947.03 ms /   874 runs   (   33.12 ms per token,    30.19 tokens per second)
llama_print_timings:       total time =   31076.19 ms /  1088 tokens


**Sepsis Definition and Protocol Overview**

Sepsis is a life-threatening organ dysfunction caused by a systemic inflammatory response to an infection. The protocol for managing sepsis in a critical care unit involves early recognition, prompt administration of antibiotics and supportive care, and careful monitoring of vital signs and organ function.

**Causes and Risk Factors**

Sepsis can be caused by any type of infection, but common culprits include pneumonia, urinary tract infections, and bloodstream infections. Risk factors for developing sepsis include age over 65, weakened immune system, recent surgery or invasive medical procedure, and underlying chronic conditions such as diabetes, heart disease, or kidney disease.

**Symptoms**

Early signs of sepsis can be subtle and may progress rapidly. Key symptoms to watch for include fever, tachycardia, tachypnea, confusion, and decreased urine output. As sepsis worsens, patients may develop septic shock, organ failure, and disseminated intravascular coagulation (DIC).

**Diagnosis**

Sepsis is diagnosed based on clinical presentation, laboratory values, and imaging studies. The Sepsis-3 definition requires the presence of systemic inflammatory response syndrome (SIRS) criteria and organ dysfunction. SIRS criteria include body temperature >38Â°C or <36Â°C, heart rate >90 beats per minute, respiratory rate >20 breaths per minute, or white blood cell count >12,000 cells/mm3 or <4,000 cells/mm3.

**Treatment and Recovery**

Early recognition and administration of antibiotics are critical for effective sepsis management. Broad-spectrum antibiotics should be administered within the first hour of diagnosis, ideally via central venous catheter. Supportive care includes mechanical ventilation, vasopressors, and fluid resuscitation as needed.

**Treatment Options**

Medical options for sepsis management include:

1. Antibiotics: Broad-spectrum antibiotics effective against common causes of sepsis, such as Pseudomonas aeruginosa and methicillin-resistant Staphylococcus aureus (MRSA).
2. Vasopressors: To maintain mean arterial pressure â¥65 mmHg and prevent hypotension.
3. Mechanical ventilation: As needed to support respiratory function.
4. Fluid resuscitation: With isotonic fluids to maintain urine output and prevent hypovolemia.

Surgical options for sepsis management include:

1. Source control: Surgical debridement or removal of the infected focus, such as a abscess or infected tissue.
2. Vascular access: Placement of central venous catheters or arterial lines for fluid resuscitation and monitoring.
3. Thoracotomy: For severe respiratory failure refractory to medical management.

**Recovery and Follow-Up**

Sepsis recovery is a complex process that requires careful monitoring of vital signs, laboratory values, and organ function. Patients may require prolonged hospitalization and rehabilitation to regain lost strength and function. Follow-up care should include regular check-ups with a primary care physician and specialists as needed.

In conclusion, the protocol for managing sepsis in a critical care unit involves early recognition, prompt administration of antibiotics and supportive care, and careful monitoring of vital signs and organ function. Prompt treatment and close monitoring can significantly improve outcomes for patients with sepsis.

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [324]:
user_prompt = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
raw_response = response_prompt(user_prompt)
display_markdown_output(raw_response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     324.51 ms /   513 runs   (    0.63 ms per token,  1580.83 tokens per second)
llama_print_timings: prompt eval time =     220.42 ms /    40 tokens (    5.51 ms per token,   181.47 tokens per second)
llama_print_timings:        eval time =   16963.93 ms /   512 runs   (   33.13 ms per token,    30.18 tokens per second)
llama_print_timings:       total time =   18026.34 ms /   552 tokens


**Symptoms of Appendicitis:**

* Sudden onset of severe pain in the lower right abdomen (often migrating to the lower back)
* Nausea and vomiting
* Loss of appetite
* Fever (usually >38Â°C/100.4Â°F)
* Abdominal tenderness and guarding (muscle tension)
* Rigidity in the abdomen
* Palpable mass in the right lower quadrant of the abdomen

**Diagnosis:**

* Physical examination findings
* Laboratory tests (e.g., complete blood count, inflammatory markers)
* Imaging studies (e.g., X-ray, CT scan) to confirm the diagnosis and assess the severity of the condition

**Treatment:**

**Medications:**

* Pain management with nonsteroidal anti-inflammatory drugs (NSAIDs) or opioids
* Antibiotics to prevent or treat secondary infections

**Surgical Procedure:**

The standard surgical procedure for appendicitis is an appendectomy, which involves the removal of the inflamed appendix. The two main types of appendectomies are:

1. Open Appendectomy: A traditional open surgery where a single incision is made in the lower right abdomen to access and remove the inflamed appendix.
2. Laparoscopic Appendectomy: A minimally invasive procedure using a laparoscope (thin tube with a camera) and specialized instruments to remove the inflamed appendix through small incisions.

**Recovery:**

After an appendectomy, patients typically spend 1-2 days in the hospital for observation and pain management. Full recovery from surgery takes about 2-4 weeks, during which time patients should avoid heavy lifting, bending, or strenuous activities.

It is important to note that appendicitis can be a medical emergency, and prompt treatment is essential to prevent complications such as perforation of the appendix, peritonitis, or sepsis. If you suspect appendicitis, seek immediate medical attention.

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [325]:
user_prompt = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
raw_response = response_prompt(user_prompt)
display_markdown_output(raw_response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     396.35 ms /   626 runs   (    0.63 ms per token,  1579.42 tokens per second)
llama_print_timings: prompt eval time =     227.27 ms /    44 tokens (    5.17 ms per token,   193.61 tokens per second)
llama_print_timings:        eval time =   20628.62 ms /   625 runs   (   33.01 ms per token,    30.30 tokens per second)
llama_print_timings:       total time =   21945.24 ms /   669 tokens


**Causes:**

1. **Alopecia Areata**: An autoimmune condition that causes sudden hair loss in patches.
2. **Telogen Effluvium**: A stress-induced condition leading to excessive hair shedding, often resulting in localized bald spots.
3. **Fungal Infections**: Malassezia or ringworm can cause hair loss in specific areas, especially if left untreated.
4. **Traction Alopecia**: Hair loss due to constant pulling or tension on the hair follicles, often seen in individuals with tight hairstyles.
5. **Inflammation**: Conditions like eczema, psoriasis, or seborrheic dermatitis can cause inflammation leading to localized hair loss.

**Symptoms:**

1. Patchy hair loss in specific areas of the scalp.
2. Hair thinning or balding in a distinct pattern.
3. Redness, itching, or scaling on the affected area.
4. Inflammation or crusting around the hair follicles.

**Diagnosis:**

1. Physical examination and medical history.
2. Blood tests to rule out underlying conditions like thyroid disorders or autoimmune diseases.
3. Skin scrapings or biopsies for fungal infections or inflammatory conditions.
4. Imaging studies (e.g., scalp biopsy) to confirm the diagnosis and assess the extent of hair loss.

**Treatment:**

1. **Topical Therapies**: Minoxidil (Rogaine) or corticosteroids can help stimulate hair growth and reduce inflammation.
2. **Oral Medications**: Anti-inflammatory drugs like prednisone or immunosuppressive medications like cyclosporine may be prescribed for severe cases.
3. **Surgical Options**: Hair transplantation or scalp reduction can be considered for extensive hair loss.
4. **Lifestyle Changes**: Reducing stress, using gentle hair care products, and avoiding tight hairstyles can help manage the condition.

**Recovery:**

1. Time required for recovery varies depending on the underlying cause and treatment approach.
2. Regular follow-ups with a dermatologist are essential to monitor progress and adjust the treatment plan as needed.
3. In some cases, hair growth may not fully recover, but treatment can help slow down or stop further hair loss.

It is important to note that sudden patchy hair loss can be distressing, and seeking professional medical advice is crucial for an accurate diagnosis and appropriate treatment.

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [326]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
raw_response = response_prompt(user_input)
display_markdown_output(raw_response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     295.10 ms /   468 runs   (    0.63 ms per token,  1585.90 tokens per second)
llama_print_timings: prompt eval time =     441.21 ms /    38 tokens (   11.61 ms per token,    86.13 tokens per second)
llama_print_timings:        eval time =   15356.26 ms /   467 runs   (   32.88 ms per token,    30.41 tokens per second)
llama_print_timings:       total time =   16549.89 ms /   505 tokens


**Depending on the severity and location of the injury, treatment options for brain tissue injuries may include:**

1. **Medical therapy:**
	* Pain management with analgesics, sedatives, or anesthetics as needed
	* Anti-seizure medications if seizures occur
	* Intravenous fluids and electrolyte replacement as needed
	* Monitoring of vital signs and neurological status
2. **Surgical intervention:**
	* Decompression surgery to relieve pressure on the brain caused by swelling or herniation
	* Craniotomy to repair skull fractures or relieve pressure on specific areas of the brain
	* Brain tumor resection if the injury is caused by a tumor
3. **Rehabilitation:**
	* Physical therapy to improve motor function and mobility
	* Occupational therapy to regain cognitive and daily living skills
	* Speech therapy to address communication and language deficits
4. **Supportive care:**
	* Monitoring for complications such as infections, seizures, or blood clots
	* Nutritional support to ensure adequate calorie and nutrient intake
	* Psychological support to address emotional and cognitive changes
5. **Experimental treatments:**
	* Stem cell therapy to promote brain repair and regeneration
	* Hyperbaric oxygen therapy to improve blood flow and reduce inflammation
	* Electrical stimulation therapies such as transcranial magnetic stimulation (TMS) or transcranial direct current stimulation (tDCS) to enhance neural plasticity and recovery.

**It is important to note that the most appropriate treatment plan will depend on the specific nature and severity of the injury, as well as the individual patient's needs and medical history. A multidisciplinary team of healthcare professionals should be involved in the decision-making process to ensure the best possible outcome for the patient.**

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [327]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
raw_response = response_prompt(user_input)
display_markdown_output(raw_response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     406.62 ms /   654 runs   (    0.62 ms per token,  1608.37 tokens per second)
llama_print_timings: prompt eval time =     234.48 ms /    44 tokens (    5.33 ms per token,   187.65 tokens per second)
llama_print_timings:        eval time =   21649.65 ms /   653 runs   (   33.15 ms per token,    30.16 tokens per second)
llama_print_timings:       total time =   23043.53 ms /   697 tokens


**Necessary Precautions:**

1. **Immobilization**: Use a splint or cast to immobilize the affected limb to prevent further injury and promote healing.
2. **Pain management**: Administer pain medication as needed, ensuring the patient is comfortable and able to rest.
3. **Wound care**: Clean and dress any open wounds to prevent infection.
4. **Hydration**: Encourage the patient to drink plenty of fluids to stay hydrated and support healing.
5. **Monitoring**: Closely monitor the patient's vital signs, particularly their pulse, breathing rate, and blood pressure.
6. **Transportation**: If necessary, carefully transport the patient to a medical facility for further evaluation and treatment.

**Treatment Steps:**

1. **Initial assessment**: Evaluate the severity of the fracture and determine if any other injuries are present.
2. **Imaging studies**: Obtain X-rays or other imaging studies to confirm the diagnosis and assess the extent of the fracture.
3. **Reducation**: If the fracture is displaced, perform reduction (realignment) of the bone to restore proper alignment.
4. **Stabilization**: Use external fixation devices, such as a cast or brace, to stabilize the affected limb and promote healing.
5. **Surgical intervention**: If necessary, perform surgery to realign and stabilize the bone.
6. **Rehabilitation**: Once the patient is stable, begin rehabilitation exercises to regain strength, mobility, and range of motion in the affected limb.

**Considerations for Care and Recovery:**

1. **Infection prevention**: Take measures to prevent infection, such as ensuring proper wound care and hygiene.
2. **Pain management**: Continue pain management as needed throughout the recovery process.
3. **Rest and recovery**: Encourage the patient to rest and avoid strenuous activities until the fracture has fully healed.
4. **Monitoring**: Regularly monitor the patient's progress, addressing any complications or concerns that arise.
5. **Follow-up appointments**: Schedule follow-up appointments with a healthcare provider to assess the patient's progress and ensure proper healing.
6. **Rehabilitation**: Encourage the patient to participate in rehabilitation exercises to regain strength, mobility, and range of motion in the affected limb.

It is important to note that the specific treatment steps and considerations for care and recovery may vary depending on the severity and location of the fracture, as well as the individual patient's needs and medical history.

## Data Preparation for RAG

### Loading the Data

In [328]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [329]:
merk_manual = '/content/drive/MyDrive/medical_diagnosis_manual.pdf'

In [330]:
pdf_loader = PyMuPDFLoader(merk_manual)
manual = pdf_loader.load()

### Data Overview

#### Checking the first 5 pages

In [331]:
for i in range(5):
    print(f"Page Number : {i+1}",end="\n")
    print(manual[i].page_content,end="\n")

Page Number : 1
ajayvmahajan@gmail.com
FZ98U1SOXE
ant for personal use by ajayvmahajan@g
shing the contents in part or full is liable
Page Number : 2
ajayvmahajan@gmail.com
FZ98U1SOXE
This file is meant for personal use by ajayvmahajan@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.
Page Number : 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ..........................................................................................................................................................................................

#### Checking the number of pages

In [332]:
len(manual)

4114

### Data Chunking

In [333]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap= 20
)

In [334]:
document_chunks = pdf_loader.load_and_split(text_splitter)

In [335]:
len(document_chunks)

8469

In [336]:
document_chunks[5000].page_content

'excessive analgesia and sedation.\n[Table 222-8. Some Causes of Agitation or Confusion in Critical Care Patients]\nPast medical history is reviewed for potential causes. Underlying liver disease suggests possible hepatic\nencephalopathy. Known substance dependency or abuse suggests a withdrawal syndrome.\nAwake, coherent patients are asked what is troubling them and are questioned specifically about pain,\ndyspnea, and previously unreported substance dependency.\nPhysical examination: O2 saturation < 90% suggests a hypoxic etiology. Low BP and urine output\nsuggest CNS hypoperfusion. Fever and tachycardia suggest sepsis or delirium tremens. Neck stiffness\nsuggests meningitis, although this finding may be difficult to demonstrate in an agitated patient. Focal\nfindings on neurologic examination suggest stroke, hemorrhage, or increased intracranial pressure (ICP).\nThe degree of agitation can be quantified using a scale such as the Riker Sedation-Agitation Scale (see\nTable 222-9) or t

In [337]:
document_chunks[5001].page_content

'evaluate because they may be highly agitated and uncomfortable despite appearing motionless. It is\ntypically necessary to allow paralysis to wear off periodically (eg, daily) so that the patient can be\nassessed.\nTesting: Identified abnormalities (eg, hypoxia, hypotension, fever) should be clarified further with\nappropriate testing. Head CT need not routinely be done unless focal neurologic findings are present or\nno other etiology is found. A bispectral index (BIS) monitor may be helpful in determining the level of\nsedation/agitation of patients under neuromuscular blockade.\nTreatment\nUnderlying conditions (eg, hypoxia, shock, drugs) should be addressed. The environment should be\noptimized (eg, darkness, quiet, and minimal sleep interruption at night) as much as is compatible with\nmedical care. Clocks, calendars, outside windows, and TV or radio programs also help connect the\npatient with the world, lessening confusion. Family presence and consistent nursing personnel may b

In [338]:
document_chunks[5002].page_content

'[Table 222-9. Riker Sedation-Agitation Scale]\npatient comfort at a lower dose of analgesic. Benzodiazepines (eg, lorazepam, midazolam) are most\ncommon, but propofol, a sedative-hypnotic drug, may be used. A common regimen for sedation is\nlorazepam 1 to 2 mg IV q 1 to 2 h or a continuous infusion at 1 to 2 mg/h if the patient is intubated. These\ndrugs pose risks of respiratory depression, hypotension, delirium, and prolonged physiologic effects in\nsome patients. Long-acting benzodiazepines such as diazepam, flurazepam, and chlordiazepoxide should\nbe avoided in the elderly. Antipsychotics with less anticholinergic effect, such as haloperidol 1 to 3 mg IV,\nmay work best when combined with benzodiazepines.\nNeuromuscular blockade: For intubated patients, neuromuscular blockade is not a substitute for\nsedation; it only removes visible manifestations of the problem (agitation) without correcting it. However,\nneuromuscular blockade may be required during tests (eg CT, MRI) or proced

In [339]:
document_chunks[5003].page_content

'Chapter 223. Cardiac Arrest\nIntroduction\nCardiac arrest is the terminal event in any fatal disorder. It may also occur suddenly (defined as within 24\nh of onset of symptoms in a previously functioning person) and, as such, occurs outside the hospital in\nabout 400,000 people/yr in the US, with a 90% mortality.\nRespiratory arrest and cardiac arrest are distinct, but without treatment, one inevitably leads to the other.\n(See also respiratory failure in Ch. 225, dyspnea on p. 1832, and hypoxia on p. 2250.)\nEtiology\nIn adults, sudden cardiac arrest results primarily from cardiac disease (of all types, but especially\ncoronary artery disease). In a significant percentage of patients, sudden cardiac arrest is the first\nmanifestation of heart disease. Other causes include circulatory shock due to noncardiac disorders\n(especially pulmonary embolism, GI hemorrhage, or trauma), ventilatory failure, and metabolic disturbance\n(including drug overdose).\nIn children, cardiac causes of su

### Embedding

In [340]:
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')

In [341]:
embedding_1 = embedding_model.embed_query(document_chunks[0].page_content)
embedding_2 = embedding_model.embed_query(document_chunks[1].page_content)

In [342]:
print("Dimension of the embedding vector ",len(embedding_1))
len(embedding_1)==len(embedding_2)

Dimension of the embedding vector  1024


True

### Vector Database

In [343]:
out_dir = 'medical_db'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

vectorstore = Chroma.from_documents(
    documents=document_chunks,
    embedding=embedding_model,
    persist_directory=out_dir
)

In [344]:
vectorstore = Chroma(
    embedding_function=embedding_model,
    persist_directory=out_dir
)

In [345]:
vectorstore.embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='thenlper/gte-large', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [346]:
vectorstore.similarity_search("symptoms appendicitis?",k=3)

[Document(metadata={'creationDate': 'D:20120615054440Z', 'trapped': '', 'keywords': '', 'creationdate': '2012-06-15T05:44:40+00:00', 'file_path': '/content/drive/MyDrive/medical_diagnosis_manual.pdf', 'moddate': '2025-05-28T15:49:29+00:00', 'format': 'PDF 1.7', 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'modDate': 'D:20250528154929Z', 'author': '', 'subject': '', 'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'page': 173, 'total_pages': 4114, 'creator': 'Atop CHM to PDF Converter', 'source': '/content/drive/MyDrive/medical_diagnosis_manual.pdf'}, page_content="Etiology\nAppendicitis is thought to result from obstruction of the appendiceal lumen, typically by lymphoid\nhyperplasia, but occasionally by a fecalith, foreign body, or even worms. The obstruction leads to\ndistention, bacterial overgrowth, ischemia, and inflammation. If untreated, necrosis, gangrene, and\nperforation occur. If the perforation is contained by the omentum, an appendiceal absc

### Retriever

In [347]:
retriever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 2} #Complete the code to pass an appropriate k value
)

In [348]:
rel_docs = retriever.get_relevant_documents(query='what are the symptoms appendicitis?')
rel_docs

[Document(metadata={'creationdate': '2012-06-15T05:44:40+00:00', 'subject': '', 'creationDate': 'D:20120615054440Z', 'file_path': '/content/drive/MyDrive/medical_diagnosis_manual.pdf', 'creator': 'Atop CHM to PDF Converter', 'author': '', 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'moddate': '2025-05-28T15:49:29+00:00', 'format': 'PDF 1.7', 'page': 173, 'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'trapped': '', 'keywords': '', 'source': '/content/drive/MyDrive/medical_diagnosis_manual.pdf', 'modDate': 'D:20250528154929Z', 'total_pages': 4114}, page_content="Etiology\nAppendicitis is thought to result from obstruction of the appendiceal lumen, typically by lymphoid\nhyperplasia, but occasionally by a fecalith, foreign body, or even worms. The obstruction leads to\ndistention, bacterial overgrowth, ischemia, and inflammation. If untreated, necrosis, gangrene, and\nperforation occur. If the perforation is contained by the omentum, an appendiceal absc

The above response is somewhat generic and is solely based on the data the model was trained on, rather than the medical manual.

Let's now provide our own context.

### System and User Prompt Template

Prompts guide the model to generate accurate responses. Here, we define two parts:

1. The system message describing the assistant's role.
2. A user message template including context and the question.

In [349]:
qna_system_message = """You are a knowledgeable clinical assistant. Based only on the provided context, answer the medical question clearly and accurately.
Use the following rules:
- Structure your answer with **bold headings** and bullet points
- Use short, clinical language suitable for physicians or informed patients
- Do not include information not found in the context
- If the context is insufficient to answer, say so explicitly
"""

In [350]:
qna_user_message_template = """Context:
{context}

Question:
{question}

Instructions:
Answer the question using only the above context. Do not make up facts. Format your answer with bullet points and bold section titles where appropriate.
"""

### Response Function

In [351]:
from IPython.display import Markdown, display

def display_markdown_response(text):
    if isinstance(text, bytes):
        text = text.decode("utf-8", errors="replace")
    else:
        text = text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')

    # Decode escape sequences
    cleaned_text = text.encode().decode("unicode_escape").strip()

    # Fix common bad encodings
    replacements = {
        "â¢": "•",
        "â": "-",
        "â": "—",
        "â": "'",
        "â": '"',
        "â": '"',
        "Bold Section Titles:": "",
        "Bullet Points:": ""
    }
    for bad, good in replacements.items():
        cleaned_text = cleaned_text.replace(bad, good)

    display(Markdown(cleaned_text))


In [352]:
def generate_rag_response(user_input,k=3,max_tokens=512,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = f"""[INST]
    {qna_system_message}

    {qna_user_message_template.format(context=context_for_query, question=user_input)}
    [/INST]"""

    # prompt = qna_system_message + '\n' + user_message

    # Generate the response
    try:
        response = llm(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

## Question Answering using RAG

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [353]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
response = generate_rag_response(user_input, max_tokens=512)
display_markdown_response(response)


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     327.26 ms /   506 runs   (    0.65 ms per token,  1546.15 tokens per second)
llama_print_timings: prompt eval time =    2287.01 ms /  1877 tokens (    1.22 ms per token,   820.72 tokens per second)
llama_print_timings:        eval time =   17933.99 ms /   505 runs   (   35.51 ms per token,    28.16 tokens per second)
llama_print_timings:       total time =   21148.69 ms /  2382 tokens


**Sepsis Management Protocol in Critical Care Units**

* **Frequent Monitoring**:
	+ Systemic pressure; CVP, PAOP, or both
	+ Pulse oximetry
	+ ABGs; blood glucose, lactate, and electrolyte levels
	+ Renal function, and possibly sublingual PCO2
	+ Urine output, a good indicator of renal perfusion
* **Fluid Resuscitation**:
	+ 0.9% saline until CVP reaches 8 mm Hg (10 cm H2O) or PAOP reaches 12 to 15 mm Hg
	+ Oliguria with hypotension is not a contraindication to vigorous fluid resuscitation
	+ Quantity of fluid required often far exceeds the normal blood volume and may reach 10 L over 4 to 12 h
* **Dopamine Therapy**:
	+ If a patient with septic shock remains hypotensive after CVP or PAOP has been raised to target levels, dopamine may be given to increase mean BP to at least 60 mm Hg
	+ However, vasoconstriction caused by higher doses of dopamine and norepinephrine poses risks of organ hypoperfusion and acidosis
* **Oxygen Therapy**:
	+ O2 is given by mask or nasal prongs
	+ Tracheal intubation and mechanical ventilation may be needed subsequently for respiratory failure
* **Supportive Care**:
	+ Provision of adequate nutrition (see p. 21)
	+ Prevention of infection, stress ulcers, and gastritis (see p. 131)
	+ Pulmonary embolism prevention (see p. 1920)
* **Blood Tests**:
	+ Routine daily blood tests to help detect problems early, including electrolytes and CBC
	+ Additional tests as needed, such as blood culture for fever, CBC after a bleeding episode

Note: The above protocol is based on the provided context and should not be considered a comprehensive or substitute for professional medical advice.

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [354]:
user_input = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response = generate_rag_response(user_input)
display_markdown_response(response)


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     261.66 ms /   409 runs   (    0.64 ms per token,  1563.10 tokens per second)
llama_print_timings: prompt eval time =    2349.41 ms /  1972 tokens (    1.19 ms per token,   839.36 tokens per second)
llama_print_timings:        eval time =   14693.11 ms /   408 runs   (   36.01 ms per token,    27.77 tokens per second)
llama_print_timings:       total time =   17760.16 ms /  2380 tokens


Sure, I can help you with that! Here's the answer based on the provided context:

**Symptoms of Appendicitis:**

* Sudden and severe pain in the abdomen (often near the navel)
* Nausea and vomiting
* Loss of appetite
* Fever
* Abdominal tenderness and guarding (muscle tension)

**Can Appendicitis be Cured via Medicine?**

No, appendicitis cannot be cured solely with medicine. Antibiotics may be used to treat any secondary infections, but they do not address the underlying inflammation of the appendix. Surgical intervention is necessary to remove the inflamed appendix and prevent further complications.

**Surgical Procedure for Treating Appendicitis:**

The preferred surgical procedure for treating appendicitis is a laparoscopic appendectomy, which involves removing the inflamed appendix through small incisions in the abdomen. This approach has been shown to have fewer complications and a faster recovery time compared to open appendectomy.

If the appendix is perforated, antibiotics may be administered before surgery to help reduce the risk of infection. In some cases, an open appendectomy may be necessary if the appendix is difficult to locate or if there are other complications present.

It's important to note that a negative appendectomy rate of 15% is considered acceptable, meaning that in about 15% of cases, the surgeon may not be able to remove the inflamed appendix due to its location or other factors. In these cases, the patient may require further treatment, such as antibiotics and supportive care, to manage any ongoing infection or inflammation.

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [355]:
user_input = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response = generate_rag_response(user_input)
display_markdown_response(response)


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     328.39 ms /   512 runs   (    0.64 ms per token,  1559.11 tokens per second)
llama_print_timings: prompt eval time =    1615.15 ms /  1231 tokens (    1.31 ms per token,   762.16 tokens per second)
llama_print_timings:        eval time =   17883.20 ms /   511 runs   (   35.00 ms per token,    28.57 tokens per second)
llama_print_timings:       total time =   20428.26 ms /  1742 tokens


Based on the provided context, here are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and the possible causes behind it:

**Effective Treatments for Sudden Patchy Hair Loss:**

1. **Medications:**
	* Corticosteroids: to reduce inflammation and suppress the immune system
	* Retinoids: to promote hair growth and prevent further hair loss
	* Immunosuppressants: to suppress the immune system and prevent autoimmune attacks on hair follicles
2. **Wigs or Hairpieces:**
	* Temporary solution for hair loss due to chemotherapy or other medical conditions
	* Can be customized to match the patient's natural hair color and style
3. **Hair Transplantation:**
	* Permanent solution for hair loss, involves transplanting healthy hair follicles from one part of the scalp to another
4. **Low-Level Laser Therapy (LLLT):**
	* Non-invasive treatment that stimulates hair growth and reduces inflammation
5. **Platelet-Rich Plasma (PRP) Therapy:**
	* Injecting platelet-rich plasma into the scalp to stimulate hair growth and reduce inflammation

**Possible Causes of Sudden Patchy Hair Loss:**

1. **Alopecia Areata:**
	* Autoimmune disorder that causes sudden patchy hair loss, often affecting the scalp and beard area
	* May affect most or all of the body (alopecia universalis)
2. **Telogen Effluvium:**
	* Condition where hair follicles enter the resting phase and shed, often due to physical or emotional stress
3. **Anagen Effluvium:**
	* Condition where hair follicles stop growing and enter the resting phase, often due to exposure to chemicals or radiation therapy
4. **Androgenetic Alopecia (Male-Pattern and Female-Pattern Hair Loss):**
	* Most common type of hair loss, caused by horm

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [356]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response = generate_rag_response(user_input)
display_markdown_response(response)


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     269.58 ms /   425 runs   (    0.63 ms per token,  1576.50 tokens per second)
llama_print_timings: prompt eval time =    2279.07 ms /  1898 tokens (    1.20 ms per token,   832.79 tokens per second)
llama_print_timings:        eval time =   15229.39 ms /   424 runs   (   35.92 ms per token,    27.84 tokens per second)
llama_print_timings:       total time =   18258.96 ms /  2322 tokens


Based on the provided context, the following treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function:

**Ensuring Adequate Airway and Breathing**

* Maintaining a reliable airway is crucial to ensure adequate oxygenation of the brain
* Endotracheal intubation may be necessary if the patient is unable to maintain their own airway

**Maintaining Adequate Blood Pressure and Oxygenation**

* Monitoring vital signs regularly to ensure stable blood pressure and oxygen saturation
* Administering fluids and medications as needed to maintain adequate blood pressure and oxygenation

**Monitoring and Treating Intracranial Pressure**

* Placing monitors to track intracranial pressure and monitor for any changes in the patient's condition
* Performing surgery if intracranial pressure is increased, such as decompressing the brain or removing hematomas

**Preventing Complications of Altered Sensorium**

* Monitoring the patient's mental status and preventing complications such as seizures, aspiration, and decubitus ulcers
* Providing appropriate medications to manage seizures, agitation, or other neurological symptoms

**Rehabilitation**

* Providing physical, occupational, and speech therapy to help the patient regain lost functions and improve their quality of life
* Monitoring the patient's progress and adjusting the rehabilitation plan as needed

It is important to note that the specific treatment plan will depend on the severity and location of the injury, as well as the individual needs of the patient. Additionally, the context does not provide information on the long-term outcomes or prognosis for patients with traumatic brain injury.

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [357]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response = generate_rag_response(user_input)
display_markdown_response(response)


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     321.67 ms /   512 runs   (    0.63 ms per token,  1591.70 tokens per second)
llama_print_timings: prompt eval time =    1273.90 ms /   578 tokens (    2.20 ms per token,   453.72 tokens per second)
llama_print_timings:        eval time =   17265.22 ms /   511 runs   (   33.79 ms per token,    29.60 tokens per second)
llama_print_timings:       total time =   19406.89 ms /  1089 tokens


Sure, I'd be happy to help! Based on the provided context, here are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip:

**Precautions:**

* Suspected open fractures require sterile wound dressings, tetanus prophylaxis, and broad-spectrum antibiotics (e.g., a 2nd-generation cephalosporin plus an aminoglycoside).
* Patients should use a cane for support.
* The cane should be moved to the lower step shortly before descending with the bad leg.

**Treatment Steps:**

1. **Immobilization:** Use a splint to immobilize the affected limb and prevent further injury or deformity.
2. **Pain Management:** Administer appropriate pain medication as needed.
3. **Wound Care:** Clean and dress any open wounds to prevent infection.
4. **Antibiotics:** Provide broad-spectrum antibiotics to prevent infection, especially for suspected open fractures.
5. **Tetanus Prophylaxis:** Administer tetanus immunoglobulin if the patient has not received a tetanus shot within the past 10 years.
6. **Monitoring:** Closely monitor the patient's vital signs, wound healing, and overall condition.
7. **Rehabilitation:** Refer the patient to physical therapy as soon as possible to facilitate recovery and prevent complications.

**Considerations for Care and Recovery:**

1. **Rest:** Encourage the patient to rest the affected limb and avoid any strenuous activities.
2. **Elevation:** Elevate the affected limb above heart level to reduce swelling and promote healing.
3. **Hydration:** Ensure the patient stays well-hydrated to support the healing process.
4. **Nutrition:** Provide a balanced diet rich in essential nutrients for optimal healing.
5. **Follow-up Care:** Schedule regular follow-up appointments with a healthcare provider

### Fine-tuning

**Query 1: What is the protocol for managing sepsis in a critical care unit?**

In [358]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
response = generate_rag_response(user_input, temperature=0.75)
display_markdown_response(response)


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     322.04 ms /   494 runs   (    0.65 ms per token,  1533.99 tokens per second)
llama_print_timings: prompt eval time =    2148.10 ms /  1778 tokens (    1.21 ms per token,   827.71 tokens per second)
llama_print_timings:        eval time =   17745.48 ms /   493 runs   (   35.99 ms per token,    27.78 tokens per second)
llama_print_timings:       total time =   20800.75 ms /  2271 tokens


**Sepsis Management Protocol in Critical Care Units**

* Monitoring:
	+ Systemic pressure (CVP, PAOP)
	+ Pulse oximetry
	+ ABGs
	+ Blood glucose, lactate, and electrolyte levels
	+ Renal function, and possibly sublingual PCO2
	+ Urine output (measured with an indwelling catheter)
* Fluid resuscitation:
	+ 0.9% saline until CVP reaches 8 mm Hg or PAOP reaches 12-15 mm Hg
	+ Oliguria with hypotension is not a contraindication to fluid resuscitation
	+ Frequent monitoring of fluid intake and output
* Vasopressor therapy:
	+ Dopamine may be given to increase mean BP to at least 60 mm Hg if CVP or PAOP targets are not achieved with fluid resuscitation
	+ Norepinephrine may be added if dopamine dose exceeds 20 g/kg/min
	+ Vasoconstriction caused by higher doses of dopamine and norepinephrine poses risks of organ hypoperfusion and acidosis
* Oxygen therapy:
	+ Given by mask or nasal prongs
	+ Tracheal intubation and mechanical ventilation may be needed subsequently for respiratory failure
* Monitoring for signs of fluid overload and pulmonary edema
* Electrolyte replacement and management of arrhythmias as needed
* Protocols for investigating alarms from complex devices
* Routine daily blood tests to help detect problems early, including:
	+ Electrolytes
	+ CBC
	+ Mg, phosphate, and Ca levels for patients with arrhythmias
	+ Liver enzymes and coagulation profiles for patients receiving TPN
	+ Other tests as needed (e.g., blood culture for fever, CBC after a bleeding episode)

Note: The above protocol is based on the provided context and should not be considered a comprehensive or substitute for medical advice.

**Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?**

In [359]:
user_input = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
response = generate_rag_response(user_input, temperature=0.75)
display_markdown_response(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     198.27 ms /   315 runs   (    0.63 ms per token,  1588.77 tokens per second)
llama_print_timings: prompt eval time =    2340.32 ms /  1972 tokens (    1.19 ms per token,   842.62 tokens per second)
llama_print_timings:        eval time =   11294.20 ms /   314 runs   (   35.97 ms per token,    27.80 tokens per second)
llama_print_timings:       total time =   14154.34 ms /  2286 tokens


Sure, I can help you with that! Here's the answer based on the provided context:

**Symptoms of Appendicitis:**

* Sudden and severe pain in the abdomen, often starting near the belly button and then moving to the lower right side
* Nausea and vomiting
* Loss of appetite
* Fever and chills
* Abdominal tenderness and guarding (muscle tension)
* Abdominal swelling

**Treatment of Appendicitis:**

* Appendectomy (surgical removal of the inflamed appendix) is the standard treatment for appendicitis
	+ Open appendectomy: an incision is made in the abdomen to manually remove the inflamed appendix
	+ Laparoscopic appendectomy: a laparoscope (thin tube with a camera and light) is inserted through small incisions to visualize the appendix, and then a specialized instrument is used to remove it
* Antibiotics may be given before or after surgery to help manage infection

It's important to note that while antibiotics can help manage the infection, they cannot cure appendicitis. Therefore, surgical removal of the inflamed appendix is necessary to treat the condition effectively.

I hope this helps! Let me know if you have any further questions.

**Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?**

In [360]:
user_input = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
response = generate_rag_response(user_input, temperature=0.75)
display_markdown_response(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     329.93 ms /   512 runs   (    0.64 ms per token,  1551.85 tokens per second)
llama_print_timings: prompt eval time =    1533.35 ms /  1231 tokens (    1.25 ms per token,   802.82 tokens per second)
llama_print_timings:        eval time =   17805.80 ms /   511 runs   (   34.85 ms per token,    28.70 tokens per second)
llama_print_timings:       total time =   20268.18 ms /  1742 tokens


Based on the provided context, here are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and the possible causes behind it:

**Effective Treatments for Sudden Patchy Hair Loss:**

1. **Medications:**
	* Corticosteroids: These medications can help reduce inflammation and suppress the immune system, which may help prevent further hair loss and promote hair growth.
	* Retinoids: Derivatives of vitamin A, these medications can help slow down hair loss and promote hair growth by increasing cell turnover and reducing scalp inflammation.
	* Immunosuppressants: These medications can help suppress the immune system and prevent it from attacking the hair follicles, which may help reduce hair loss and promote hair growth.
2. **Topical Treatments:**
	* Minoxidil (Rogaine): A solution that is applied directly to the scalp, minoxidil can help stimulate hair growth and slow down hair loss.
	* Finasteride (Propecia): A medication that is available in pill form, finasteride can help prevent hair loss by blocking the production of dihydrotestosterone (DHT), a hormone that contributes to hair loss.
3. **Surgical Solutions:**
	* Hair transplantation: This procedure involves transplanting healthy hair follicles from one part of the scalp to another, and can help restore hair growth in balding areas.

**Possible Causes of Sudden Patchy Hair Loss:**

1. **Autoimmune Disorders:**
	* Alopecia areata: An autoimmune disorder that causes sudden patchy hair loss, alopecia areata can affect any hairy area of the body and is thought to be triggered by environmental factors.
2. **Hormonal Imbalances:**
	* Androgenetic alopecia (male-pattern and female-pattern hair loss): The most common type of hair loss, androgenetic alopecia is caused by a combination of genetics and hormonal imbalances, particularly an excess of dihydrotestosterone (DHT).

**Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?**

In [361]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
response = generate_rag_response(user_input, temperature=0.75)
display_markdown_response(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     213.49 ms /   327 runs   (    0.65 ms per token,  1531.72 tokens per second)
llama_print_timings: prompt eval time =    2395.19 ms /  1898 tokens (    1.26 ms per token,   792.42 tokens per second)
llama_print_timings:        eval time =   11738.36 ms /   326 runs   (   36.01 ms per token,    27.77 tokens per second)
llama_print_timings:       total time =   14697.81 ms /  2224 tokens


Sure! Here's the answer to the medical question based on the provided context:

**Initial Treatment for Traumatic Brain Injury (TBI)**

* Ensure a reliable airway and maintain adequate ventilation, oxygenation, and blood pressure
* Manage intracranial pressure and prevent complications such as increased intracranial pressure or herniation
* Monitor and treat any contusions, intracerebral or subarachnoid hemorrhage, or epidural or subdural hematomas
* Provide appropriate neurological and neurosurgical consultation

**Surgical Interventions for TBI**

* Place monitors to track and treat intracranial pressure
* Decompress the brain if intracranial pressure is increased
* Remove intracranial hematomas or blood clots

**Other Recommended Treatments for TBI**

* Maintaining adequate brain perfusion and oxygenation in the first few days after injury
* Preventing complications of altered sensorium
* Managing the consequences of concussion, including loss of consciousness or memory
* Providing appropriate rehabilitation to address cognitive, motor, and behavioral deficits

**Note: The specific treatment plan for TBI will depend on the severity and location of the injury, as well as the individual patient's needs and medical history.**

**Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?**

In [362]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
response = generate_rag_response(user_input, temperature=0.75)
display_markdown_response(response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     251.66 ms /   389 runs   (    0.65 ms per token,  1545.72 tokens per second)
llama_print_timings: prompt eval time =     839.09 ms /   578 tokens (    1.45 ms per token,   688.84 tokens per second)
llama_print_timings:        eval time =   13041.76 ms /   388 runs   (   33.61 ms per token,    29.75 tokens per second)
llama_print_timings:       total time =   14514.08 ms /   966 tokens


**Splinting and Immobilization**

* Suspected open fractures require sterile wound dressings, tetanus prophylaxis, and broad-spectrum antibiotics (e.g., a 2nd-generation cephalosporin plus an aminoglycoside)
* Non-displaced closed fractures can be treated with immobilization using a splint or cast

**Wound Care and Infection Prevention**

* Keep the wound clean and dry to prevent infection
* Change dressings daily or as often as recommended by a healthcare professional
* Monitor for signs of infection, such as redness, swelling, or increased pain

**Pain Management**

* Use nonsteroidal anti-inflammatory drugs (NSAIDs) and opioids as needed to manage pain
* Consider using a cane for ambulation to reduce weight-bearing on the affected leg

**Rehabilitation and Recovery**

* Follow a rehabilitation plan that includes physical therapy and exercises to restore strength, flexibility, and range of motion in the affected limb
* Consider using a brace or orthosis to support the affected limb during recovery
* Monitor progress and adjust the rehabilitation plan as needed to ensure proper healing and minimize complications

**Considerations for Care and Recovery**

* Provide adequate rest and avoid overexertion during the recovery period
* Monitor for signs of complications, such as infection, nerve damage, or poor wound healing
* Follow up with a healthcare professional regularly to ensure proper healing and address any concerns or complications that arise.

## Output Evaluation

Let us now use the LLM-as-a-judge method to check the quality of the RAG system on two parameters - retrieval and generation.

We are using the same Mistral model for evaluation, so basically here the llm is rating itself on how well he has performed in the task.

In [363]:
groundedness_rater_system_message = """You are a medical quality evaluator. You are given a context, a question, and an answer.

Your task is to evaluate whether the answer is **well grounded** in the context:
- The answer should only include facts supported by the context.
- It should not hallucinate or invent facts not present in the context.

Rate the groundedness as one of the following:
- HIGH: Answer is fully supported by the context
- MEDIUM: Some minor content is unsupported but most is grounded
- LOW: Many details are not supported or are fabricated
"""

In [364]:
relevance_rater_system_message = """You are a medical quality evaluator. You are given a context, a question, and an answer.

Your task is to evaluate how **relevant** the answer is to the question:
- Does it directly address what was asked?
- Does it stay focused or go off-topic?

Rate the relevance as one of the following:
- HIGH: Directly and fully answers the question
- MEDIUM: Partially answers, with some unrelated or missing info
- LOW: Mostly irrelevant or off-topic
"""

In [365]:
user_message_template = """Context:
{context}

Question:
{question}

Answer:
{answer}

Rate the answer based on the above context and instructions.
"""

In [366]:
def generate_ground_relevance_response(user_input,k=2,max_tokens=512,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    #llm.reset()
    response = llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    answer =  response["choices"][0]["text"]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    #llm_reset()
    response_1 = llm(
            prompt=groundedness_prompt,
            max_tokens=200,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    #llm_reset()
    response_2 = llm(
            prompt=relevance_prompt,
            max_tokens=200,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            echo=False
            )

    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

In [367]:
from IPython.display import Markdown, display

def display_rag_evaluation(ground_text, rel_text):
    def extract_score_and_reason(text):
        lines = text.strip().split("\n")
        score_line = next((line for line in lines if "rate" in line.lower() and ("high" in line.lower() or "medium" in line.lower() or "low" in line.lower())), "")
        score = "HIGH" if "high" in score_line.lower() else "MEDIUM" if "medium" in score_line.lower() else "LOW"
        return score.upper(), text.strip()

    grounded_score, grounded_reason = extract_score_and_reason(ground_text)
    relevance_score, relevance_reason = extract_score_and_reason(rel_text)

    display(Markdown(f"""
### RAG Evaluation Results

** Groundedness:** `{grounded_score}`
{grounded_reason}

---

** Relevance:** `{relevance_score}`
{relevance_reason}
"""))


**Query 1: What is the protocol for managing sepsis in a critical care unit?**

In [368]:
ground,rel = generate_ground_relevance_response(user_input="What is the protocol for managing sepsis in a critical care unit?")
display_rag_evaluation(ground, rel)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     337.37 ms /   512 runs   (    0.66 ms per token,  1517.61 tokens per second)
llama_print_timings: prompt eval time =    2250.97 ms /  1874 tokens (    1.20 ms per token,   832.53 tokens per second)
llama_print_timings:        eval time =   18540.43 ms /   511 runs   (   36.28 ms per token,    27.56 tokens per second)
llama_print_timings:       total time =   21754.99 ms /  2385 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =      72.92 ms /   115 runs   (    0.63 ms per token,  1577.18 tokens per second)
llama_print_timings: prompt eval time =    2878.52 ms /  2408 tokens (    1.20 ms per token,   836.54 tokens per second)
llama_print_timings:        eval time =    4073.34 ms /   114 runs   (   35.73 ms per token,    27.99 tokens per second)
llama_print_timings:       to


### RAG Evaluation Results

** Groundedness:** `HIGH`  
Based on the provided context and instructions, I would rate the answer as HIGH in terms of groundedness. The answer provides specific and detailed information about the protocol for managing sepsis in a critical care unit, which is fully supported by the context. The answer does not hallucinate or invent facts not present in the context, and it accurately summarizes the key points from the provided text. Overall, the answer is well-grounded and provides a comprehensive overview of the sepsis management protocol in critical care units.

---

** Relevance:** `HIGH`  
Based on the context and instructions, I would rate the answer as HIGH in relevance. The answer directly addresses the question by providing a detailed protocol for managing sepsis in a critical care unit, including monitoring and testing, fluid resuscitation, vasoactive medications, oxygen therapy, and supportive care. The answer is well-structured and includes specific examples and references to the Merck Manual of Diagnosis & Therapy, 19th Edition, which adds credibility to the information provided. Overall, the answer fully addresses the question and provides a comprehensive overview of the protocol for managing sepsis in a critical care unit.


**Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?**

In [369]:
ground,rel = generate_ground_relevance_response(user_input="What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?")
display_rag_evaluation(ground, rel)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     284.75 ms /   441 runs   (    0.65 ms per token,  1548.72 tokens per second)
llama_print_timings: prompt eval time =    2626.94 ms /  2066 tokens (    1.27 ms per token,   786.47 tokens per second)
llama_print_timings:        eval time =   15923.49 ms /   440 runs   (   36.19 ms per token,    27.63 tokens per second)
llama_print_timings:       total time =   19335.21 ms /  2506 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =      90.42 ms /   145 runs   (    0.62 ms per token,  1603.57 tokens per second)
llama_print_timings: prompt eval time =    2989.71 ms /  2531 tokens (    1.18 ms per token,   846.57 tokens per second)
llama_print_timings:        eval time =    5164.05 ms /   144 runs   (   35.86 ms per token,    27.89 tokens per second)
llama_print_timings:       to


### RAG Evaluation Results

** Groundedness:** `HIGH`  
Based on the provided context and instructions, I would rate the answer as HIGH in terms of groundedness. The answer provides a clear and accurate summary of the symptoms and treatment options for appendicitis, including the surgical procedures that can be used to treat the condition.

The answer is well-supported by the context, which provides detailed information about the diagnosis and treatment of appendicitis. The answer does not hallucinate or invent facts not present in the context, and it does not contain any major content that is unsupported or fabricated.

Overall, the answer demonstrates a high level of groundedness and is well-supported by the provided context.

---

** Relevance:** `HIGH`  
Based on the provided context and instructions, I would rate the answer as HIGH in relevance. The answer directly addresses the question by providing a detailed description of the symptoms of appendicitis and the surgical procedures used to treat the condition. The information is accurate and comprehensive, covering both laparoscopic and open appendectomy options. The answer also mentions the use of antibiotics in cases where surgery is not possible, which is relevant to the question. Overall, the answer is well-structured and easy to follow, making it a high-relevance response to the question.


**Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?**

In [370]:
ground,rel = generate_ground_relevance_response(user_input="What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",max_tokens=370)
display_rag_evaluation(ground, rel)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     231.80 ms /   370 runs   (    0.63 ms per token,  1596.18 tokens per second)
llama_print_timings: prompt eval time =    1582.24 ms /  1324 tokens (    1.20 ms per token,   836.79 tokens per second)
llama_print_timings:        eval time =   12863.31 ms /   369 runs   (   34.86 ms per token,    28.69 tokens per second)
llama_print_timings:       total time =   15042.53 ms /  1693 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     105.40 ms /   160 runs   (    0.66 ms per token,  1518.01 tokens per second)
llama_print_timings: prompt eval time =    2093.23 ms /  1719 tokens (    1.22 ms per token,   821.22 tokens per second)
llama_print_timings:        eval time =    5722.57 ms /   159 runs   (   35.99 ms per token,    27.78 tokens per second)
llama_print_timings:       to


### RAG Evaluation Results

** Groundedness:** `MEDIUM`  
Based on the provided context and instructions, I would rate the answer as MEDIUM. The answer provides some information that is supported by the context, such as the possible causes of sudden patchy hair loss (Alopecia Areata) and effective treatments for addressing it. However, the answer also includes some information that is not fully supported by the context, such as the mention of hormonal imbalance as a possible cause of Androgenetic Alopecia, which is not explicitly stated in the provided text. Additionally, the answer could benefit from more specific examples or references to support the listed treatments and causes. Overall, while the answer shows some understanding of the topic, it could be improved with more careful attention to detail and supporting evidence.

---

** Relevance:** `MEDIUM`  
Based on the provided context and instructions, I would rate the answer as MEDIUM. The answer partially addresses the question by providing information on effective treatments and possible causes of sudden patchy hair loss (alopecia areata), androgenetic alopecia (male-pattern and female-pattern hair loss), and hair loss due to chemotherapy. However, the answer goes off-topic by mentioning hormonal imbalance and genetic predisposition as possible causes of hair loss, which are not directly related to the question.

Here's a breakdown of the relevance rating:

* High relevance: The answer provides information on effective treatments for alopecia areata and androgenetic alopecia, which are directly related to the question.
* Medium relevance: The answer mentions possible causes of hair loss, such as hormonal imbalance and genetic predisposition, but these


**Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?**

In [371]:
ground,rel = generate_ground_relevance_response(user_input="What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?")
display_rag_evaluation(ground, rel)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     195.92 ms /   307 runs   (    0.64 ms per token,  1566.96 tokens per second)
llama_print_timings: prompt eval time =    2352.84 ms /  1991 tokens (    1.18 ms per token,   846.21 tokens per second)
llama_print_timings:        eval time =   10941.26 ms /   306 runs   (   35.76 ms per token,    27.97 tokens per second)
llama_print_timings:       total time =   13805.18 ms /  2297 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     111.94 ms /   175 runs   (    0.64 ms per token,  1563.32 tokens per second)
llama_print_timings: prompt eval time =    2815.00 ms /  2322 tokens (    1.21 ms per token,   824.87 tokens per second)
llama_print_timings:        eval time =    6354.22 ms /   174 runs   (   36.52 ms per token,    27.38 tokens per second)
llama_print_timings:       to


### RAG Evaluation Results

** Groundedness:** `HIGH`  
Based on the provided context and instructions, I would rate the answer as HIGH in terms of groundedness. The answer provides a comprehensive list of treatments for traumatic brain injury (TBI) that are supported by the content of the provided chapter. The answer includes specific details such as ensuring a reliable airway, maintaining adequate ventilation and oxygenation, and surgery to place monitors or remove hematomas, which are all directly related to the management of TBI as described in the context. Additionally, the answer mentions rehabilitation, management of complications, and medications to control symptoms, which are also consistent with the content of the chapter. Overall, the answer is well-supported by the provided context and does not hallucinate or invent facts not present in the context.

---

** Relevance:** `HIGH`  
Based on the provided context and instructions, I would rate the answer as HIGH in relevance. The answer directly addresses the question by providing a comprehensive list of treatments recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function. The answer is well-structured and includes specific details about the initial treatment, early management, and subsequent treatment options. Overall, the answer fully addresses the question and provides accurate information based on the provided context.


**Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?**

In [372]:
ground,rel = generate_ground_relevance_response(user_input="What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?",max_tokens=370)
display_rag_evaluation(ground, rel)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     200.33 ms /   312 runs   (    0.64 ms per token,  1557.43 tokens per second)
llama_print_timings: prompt eval time =     904.80 ms /   671 tokens (    1.35 ms per token,   741.60 tokens per second)
llama_print_timings:        eval time =   10459.48 ms /   311 runs   (   33.63 ms per token,    29.73 tokens per second)
llama_print_timings:       total time =   11840.67 ms /   982 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1761.86 ms
llama_print_timings:      sample time =     134.45 ms /   200 runs   (    0.67 ms per token,  1487.58 tokens per second)
llama_print_timings: prompt eval time =    1159.34 ms /  1007 tokens (    1.15 ms per token,   868.60 tokens per second)
llama_print_timings:        eval time =    6818.78 ms /   199 runs   (   34.27 ms per token,    29.18 tokens per second)
llama_print_timings:       to


### RAG Evaluation Results

** Groundedness:** `HIGH`  
Based on the provided context and instructions, I would rate the answer as HIGH in terms of groundedness. The answer provides a comprehensive list of necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and all of the information is supported by the context.

The answer includes details such as the use of sterile wound dressings, tetanus prophylaxis, broad-spectrum antibiotics, immobilization with a splint or cast, pain management with analgesics, rest, and avoidance of strenuous activities to allow the fracture to heal. All of these details are directly supported by the context, which emphasizes the importance of sterile wound dressings, tetanus prophylaxis, and broad-spectrum antibiotics for suspected open fractures, as well as the use of a cane for amb

---

** Relevance:** `HIGH`  
Based on the given context and instructions, I would rate the answer as HIGH in relevance. The answer directly addresses the question by providing necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, including splinting and immobilization, tetanus prophylaxis, broad-spectrum antibiotics, pain management, rest, and rehabilitation. The answer also mentions the use of a cane for ambulation, which is relevant to the question.

The answer stays focused on the topic and does not go off-topic or include unrelated information. It provides a clear and concise summary of the necessary precautions and treatment steps for a person with a fractured leg, which is directly related to the question asked. Therefore, I would rate the answer as HIGH in relevance.


## Actionable Insights and Business Recommendations

**LLMs Can Definitely Help with Clinical Questions**
When you guide the model with healthcare-specific prompts, it does a much better job — answers are clearer, more accurate, and more useful.

**Prompting Makes a Big Difference**
Telling the model exactly how to structure its answers (like using bullet points or medical terms) really helps it respond in a way that feels more polished and helpful.

**Decent Performance on Common Medical Topics**
For well-known conditions, the model’s answers lined up pretty well with standard medical practices — good enough for patient education or quick references.

**Generic Models Give Generic Answers**
If you don’t give them the right context, models like LLaMA tend to be vague. They don’t really handle specific or complex clinical questions on their own.

**Good Prompts Help — But Only to a Point**
Crafting smart prompts improves things a lot, but without adding real context (like actual medical text), there’s still a limit to how specific the answers can get.

**RAG Changes the Game**
Using Retrieval-Augmented Generation (RAG) — where the model pulls from actual documents like PDFs or manuals — makes the answers way more accurate and grounded in real info.

**Ready to Be Put to Work**
This setup is already a strong starting point for building a specialized assistant. It could be used for helping patients, training staff, or even answering internal questions.

<font size=6 color='blue'>Power Ahead</font>
___